# 動画の作成

In [ ]:
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import ipywidgets
import os
import cv2

In [ ]:
DIR = "./"
TASK = ['camera','run']

task_widget = ipywidgets.Dropdown(options=TASK, description='task')
datasets_widget = ipywidgets.Dropdown(description='dataset')
make_button = ipywidgets.Button(description='Make movie')
video_name_widget = ipywidgets.Text(description='動画名')

In [ ]:
l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

In [ ]:
from torch2trt import TRTModule

model_trt = TRTModule()

In [ ]:
model_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_time_widget = ipywidgets.Label(description='作成日時：')
load_button = ipywidgets.Button(description='Load')

In [ ]:
import glob
import cv2 as cv
import sys
import subprocess
import datetime
import torch

IMG_WIDTH=224
IMG_HEIGHT=224
from utils import preprocess
model_load = False
def make_movie(change):
    global model_trt,task_widget, datasets_widget, video_name_widget
    
    if model_load == False:
        write_log("先に、モデルを読み込んでください。")
        return
    
    if video_name_widget.value == "":
        write_log("動画名を指定してください。")
        return
    
    write_log("動画を作成します。")
    path = "./video/"
    if not os.path.exists(path):
        subprocess.call(['mkdir', '-p', path])
    output = path + video_name_widget.value + ".mp4"
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    outfh = cv.VideoWriter(output, fourcc, 10, (224, 224))
    for file_name in sorted(glob.glob(task_widget.value + '/' + datasets_widget.value + '/xy/*.jpg')):

        img = cv.imread(file_name)
        
        image = preprocess(img).half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        result_x = float(output[0])
        result_y = float(output[1])
        result_x = int(IMG_WIDTH * (result_x / 2.0 + 0.5))
        result_y = int(IMG_HEIGHT * (result_y / 2.0 + 0.5))    
        img = cv2.circle(img, (int(result_x), int(result_y)), 8, (255, 0, 0), 3)
        
        # Speed
        result_speed = output[3]
        result_speed = int(IMG_WIDTH * (result_speed / 2.0 + 0.5))
        if result_speed > 224:
            result_speed = 224
        elif result_speed < 0:
            result_speed = 0
        img = cv2.line(img,(218,0),(218,224),(0,0,0),5)
        img = cv2.line(img,(219,224-result_speed),(219,224),(0,140,255),3)
        img = cv2.putText(img,"speed:"+str(result_speed),(160,215),cv2.FONT_HERSHEY_SIMPLEX,0.3,(255,255,255))
      
        write_log(file_name)
        outfh.write(img)
    outfh.release()
    
def model_list(change):
    global model_widget
    try:
        files = glob.glob('./model_trt/*.pth', recursive=True)
        model_widget.options = files
        ts = os.path.getctime(files[0])
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        model_time_widget.value = f'作成日時：{s}'
    except:
        model_widget.options = []
model_list("list")

def change_file(change):
    file = model_widget.value
    ts = os.path.getctime(file)
    d = datetime.datetime.fromtimestamp(ts)
    s = d.strftime('%Y-%m-%d %H:%M:%S')
    model_time_widget.value = f'作成日時：{s}'
model_widget.observe(change_file, names='value')
    
def load_model(change):
    global model_trt,model_widget,model_load
    try:
        write_log(model_widget.value + "の読込を実行します(初回は時間がかかります)。")
        model_trt.load_state_dict(torch.load(model_widget.value))
        write_log(model_widget.value + "の読込に成功しました。")
        model_load = True
    except:
        model_load = False
        write_log("[Error]" + model_widget.value + "の読込に失敗しました。")

load_button.on_click(load_model)
make_button.on_click(make_movie)

In [ ]:
def change_task(change):
    global DIR, task_widget, datasets_widget
    try:        
        path = DIR + task_widget.value + "/"
        files = os.listdir(path)
        dirs = [f for f in files if os.path.isdir(os.path.join(path, f))]
        dirs = [f for f in files if f != ".ipynb_checkpoints"]
        dirs = sorted(dirs)
        datasets_widget.options = dirs
    except:
        #write_log(path + "が存在していません。")
        datasets_widget.options = []
task_widget.observe(change_task, names='value')
change_task(TASK[0])
        
movie_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([model_widget,model_time_widget,load_button]),
    ipywidgets.HBox([datasets_widget,task_widget]),
    ipywidgets.HBox([video_name_widget,make_button]),
    process_widget
])
display(movie_collection_widget)